##### Mathys BRONNEC, Lisa CHARUEL, Jules DEPLANCHON

In [1]:
doughLength=500

import pandas as pd
#creation of a Biscuit dataset (first )
dataset = {'length':[4,8,2,5],'value':[6,12,1,8],'maximum allowed defects':[{'a': 4, 'b': 2, 'c': 3},{'a': 5, 'b': 4, 'c': 4},{'a': 1, 'b': 2, 'c': 1},{'a': 2, 'b': 3, 'c': 2}]}

df = pd.DataFrame(dataset)
display(df)

#import of the defects in a dataframe
defects = pd.read_csv('defects.csv')
defects.sort_values(by=['x'],inplace=True)
display(defects)

#creation of a class biscuit
class Biscuit : 
    def __init__(self, length, value, maximum_allowed_defects) :
        self.length = length
        self.value = value
        self.maximum_allowed_defects = maximum_allowed_defects

    def __str__(self) :
        if self.value==0 :
            return f"empty space"
        return f"biscuit of length {self.length} and value {self.value}"

    def empty_space() :
        return Biscuit(1,0,{'a':999,'b':999,'c':999})

#creation of a class dough
class dought : 
    def __init__(self, biscuits) :
        self.biscuits = biscuits
    
    def next_x(self) :
        return sum([b.length for b in self.biscuits])
    
    def number_of_defects(self,x,length) :
        interval=defects[(defects['x']>=x) & (defects['x']<x+length)]
        return {'a':interval[interval['class']=='a'].count().astype(int)[0],
                'b':interval[interval['class']=='b'].count().astype(int)[0],
                'c':interval[interval['class']=='c'].count().astype(int)[0]}

    def canbeplaced(self, biscuit) :
        if biscuit.length <= doughLength-self.next_x() :
            number_of_defects=self.number_of_defects(self.next_x(),biscuit.length)
            if (number_of_defects['a'] <= biscuit.maximum_allowed_defects['a'] and number_of_defects['b'] <= biscuit.maximum_allowed_defects['b'] and number_of_defects['c'] <= biscuit.maximum_allowed_defects['c']) :
                return True
        return False
    
    def Value(self) :
        return sum([b.value for b in self.biscuits])

    def leftemptyspace(self) :
        self.biscuits.append(Biscuit.empty_space())

    def place(self, biscuit) :
        if self.canbeplaced(biscuit) :
            self.biscuits.append(biscuit)
            return True
        return False

    def printDough(self) :
        x=0
        for b in self.biscuits :
            print(f"{b} on the interval [{x},{x+b.length}[")
            x+=b.length
        print(f"\n\ntotal value : {self.Value()}, number of biscuits : {len(self.biscuits)-len([b for b in self.biscuits if b.value==0])}")

#instantiation of the 4 types of biscuits with the biscuit class
Biscuit0 = Biscuit(4,6,{'a':4,'b':2,'c':3})
Biscuit1 = Biscuit(8,12,{'a':5,'b':4,'c':4})
Biscuit2 = Biscuit(2,1,{'a':1,'b':2,'c':1})
Biscuit3 = Biscuit(5,8,{'a':2,'b':3,'c':2})




,length,value,maximum allowed defects
0,4,6,"{'a': 4, 'b': 2, 'c': 3}"
1,8,12,"{'a': 5, 'b': 4, 'c': 4}"
2,2,1,"{'a': 1, 'b': 2, 'c': 1}"
3,5,8,"{'a': 2, 'b': 3, 'c': 2}"


,x,class
479,0.700561,a
141,0.848185,a
142,1.769825,a
493,2.207230,b
125,2.313038,a
...,...,...
219,496.411441,a
449,496.429679,b
243,497.370667,a
246,498.455612,c


# Informed search : Greedy algorithm implementation

This Algoritm tries to place the highest possible value by unit biscuit. If it can't place any biscuit, it places an empty space.

In [2]:
roll = dought([])
biscuits = [Biscuit0,Biscuit1,Biscuit2,Biscuit3]
#sort biscuits by value/length ratio (score/unit)
biscuits.sort(key=lambda x: x.value/x.length, reverse=True)

while roll.next_x()<doughLength: #while there is still space on the roll
    i=0 #set the list index to 0
    placed=False #bool to stop the loop if a biscuit was placed or place an empty space if no biscuit was placed

    #try to place the biscuit with the highest score/unit
    while i<len(biscuits) and placed==False:
        placed=roll.place(biscuits[i])
        i+=1 #increment the index to try the next biscuit

    #if no biscuit was placed, add an empty space
    if i==len(biscuits) and placed==False:
        roll.leftemptyspace()

roll.printDough() #print the result

biscuit of length 8 and value 12 on the interval [0,8[
biscuit of length 8 and value 12 on the interval [8,16[
biscuit of length 5 and value 8 on the interval [16,21[
biscuit of length 4 and value 6 on the interval [21,25[
biscuit of length 4 and value 6 on the interval [25,29[
biscuit of length 5 and value 8 on the interval [29,34[
biscuit of length 2 and value 1 on the interval [34,36[
biscuit of length 4 and value 6 on the interval [36,40[
biscuit of length 4 and value 6 on the interval [40,44[
biscuit of length 4 and value 6 on the interval [44,48[
biscuit of length 2 and value 1 on the interval [48,50[
biscuit of length 4 and value 6 on the interval [50,54[
biscuit of length 5 and value 8 on the interval [54,59[
biscuit of length 5 and value 8 on the interval [59,64[
biscuit of length 4 and value 6 on the interval [64,68[
biscuit of length 5 and value 8 on the interval [68,73[
biscuit of length 4 and value 6 on the interval [73,77[
biscuit of length 5 and value 8 on the interval [

## Constraint satisfaction algorithm implementation:

In [7]:
from ortools.sat.python import cp_model

biscuits = [Biscuit0,Biscuit1,Biscuit2,Biscuit3]

#number of defects for a biscuit starting at position x
def number_of_defects(x,biscuit) :
        length = biscuit.length
        interval=defects[(defects['x']>=x) & (defects['x']<x+length)]
        return {'a':interval[interval['class']=='a'].count().astype(int)[0],
                'b':interval[interval['class']=='b'].count().astype(int)[0],
                'c':interval[interval['class']=='c'].count().astype(int)[0]}

#initialize a dictionary with all the possible defects for each biscuit at each position
defects_by_position = {}

#fill the dictionary
for i, biscuit in enumerate(biscuits):
    defects_by_position[biscuit] = {}
    for start_position in range(doughLength - biscuit.length + 1):
        defects_by_position[biscuit][start_position] = number_of_defects(start_position, biscuit)

#return the number of defects for a biscuit at a certain position
def get_defects_at_position(position, biscuit):
    return defects_by_position[biscuit][position]

#Create the model.
model = cp_model.CpModel()

#Each variable indicates if a biscuit starts at a certain position
start_vars = {}
for i, biscuit in enumerate(biscuits):
    for j in range(doughLength - biscuit.length + 1):
        start_vars[(i, j)] = model.NewBoolVar(f'start_biscuit_{i}_at_{j}')
        
#Overlapping constraint
for j in range(doughLength):
    model.Add(sum(start_vars[(i, k)] for i, biscuit in enumerate(biscuits) 
                 for k in range(max(0, j - biscuit.length + 1), min(j + 1, doughLength - biscuit.length + 1))) <= 1)

#Defect constraints (basically, tells all the positions a biscuit can't have for each biscuit)
for i, biscuit in enumerate(biscuits):
    for j in range(doughLength - biscuit.length + 1):
        for defect_type in ['a', 'b', 'c']:
            if (defects_by_position[biscuit][j][defect_type] > biscuit.maximum_allowed_defects[defect_type]):
                model.Add(start_vars[(i, j)] == 0)
                break

#Maximizing the sum of biscuit values
model.Maximize(sum(start_vars[(i, j)] * biscuits[i].value for i, biscuit in enumerate(biscuits) for j in range(doughLength - biscuit.length + 1)))

#Solve the model.
solver = cp_model.CpSolver()
status = solver.Solve(model)

#print the status of the solution
print(status==cp_model.OPTIMAL)

#store the solution in a dataframe
biscuitpositions = {"position":[], "biscuit":[]}

if status == cp_model.OPTIMAL or status == cp_model.FEASIBLE:
    for i, biscuit in enumerate(biscuits):
        for j in range(doughLength - biscuit.length + 1):
            if solver.Value(start_vars[(i, j)]):
                biscuitpositions["position"].append(j)
                biscuitpositions["biscuit"].append(biscuit)
    print(f'Optimal objective value: {solver.ObjectiveValue()}')
else:
    print("No solution was found.")

biscuitpositions = pd.DataFrame(biscuitpositions)
biscuitpositions.sort_values(by=['position'],inplace=True)
biscuitpositions.reset_index(drop=True)


def canbeplaced(biscuit,position) :
        if biscuit.length <= doughLength-position :
            number_of_defectss=number_of_defects(position,biscuit)
            if (number_of_defectss['a'] <= biscuit.maximum_allowed_defects['a'] and number_of_defectss['b'] <= biscuit.maximum_allowed_defects['b'] and number_of_defectss['c'] <= biscuit.maximum_allowed_defects['c']) :
                return True
        print(f"biscuit {biscuit} can't be placed !")
        return False

#sort the biscuits by position
biscuitpositions.sort_values(by=['position'],inplace=True)

#fill the empty spaces with "empty spaces biscuits" (biscuits with value 0 and length 1)
def fillemptyspace(biscuitpositions) :
    pos=0
    newrows={"position":[],"biscuit":[]}
    for position,biscuit in zip(biscuitpositions['position'],biscuitpositions['biscuit']):
        if pos<position :
            for i in range(pos,position):
                newrows['position'].append(i)
                newrows['biscuit'].append(Biscuit.empty_space())
        pos=position+biscuit.length

    newrows=pd.DataFrame(newrows)
    biscuitpositions=pd.concat([biscuitpositions,newrows],ignore_index=True)

    return biscuitpositions

#use the function to fill the empty spaces
biscuitpositions=fillemptyspace(biscuitpositions)
biscuitpositions.sort_values(by=['position'],inplace=True)
        
#Print the solution found by the solver
#We use the dought class to check if the solution is valid
#If the solution is not valid, the optimal objective value and the value will differ, else they will be the same
roll=dought([])
for biscuit in biscuitpositions['biscuit']:
    roll.place(biscuit)
roll.printDough()

True
Optimal objective value: 760.0
biscuit of length 8 and value 12 on the interval [0,8[
biscuit of length 8 and value 12 on the interval [8,16[
biscuit of length 5 and value 8 on the interval [16,21[
empty space on the interval [21,22[
biscuit of length 4 and value 6 on the interval [22,26[
biscuit of length 5 and value 8 on the interval [26,31[
biscuit of length 5 and value 8 on the interval [31,36[
biscuit of length 4 and value 6 on the interval [36,40[
empty space on the interval [40,41[
biscuit of length 5 and value 8 on the interval [41,46[
biscuit of length 4 and value 6 on the interval [46,50[
biscuit of length 4 and value 6 on the interval [50,54[
biscuit of length 5 and value 8 on the interval [54,59[
biscuit of length 4 and value 6 on the interval [59,63[
biscuit of length 4 and value 6 on the interval [63,67[
biscuit of length 4 and value 6 on the interval [67,71[
biscuit of length 4 and value 6 on the interval [71,75[
biscuit of length 4 and value 6 on the interval [75,7

### We have a better solution than with the greedy algorithm and it may be the best solution since the status of the model is "OPTIMAL". There are multiple solutions that give us the score of 760 but we don't think there is a solution giving a better score.